In [1]:
import sys
sys.path.append('/Users/dan/Code/LLM/llm_researcher')

In [9]:
from langchain_community.retrievers import ArxivRetriever
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders.html_bs import BSHTMLLoader
from langchain_community.document_loaders import WebBaseLoader
from bs4 import BeautifulSoup
import uuid
import requests
import re
import os

In [4]:
link="https://arxiv.org/abs/2305.04091"
def arxiv_scrape(link):
    query = link.split("/")[-1]
    retriever = ArxivRetriever(load_max_docs=2, doc_content_chars_max=None)
    docs = retriever.invoke(query)
    # Just pulling the abstract
    return docs[0].page_content

arxiv_scrape(link)

'Large language models (LLMs) have recently been shown to deliver impressive\nperformance in various NLP tasks. To tackle multi-step reasoning tasks,\nfew-shot chain-of-thought (CoT) prompting includes a few manually crafted\nstep-by-step reasoning demonstrations which enable LLMs to explicitly generate\nreasoning steps and improve their reasoning task accuracy. To eliminate the\nmanual effort, Zero-shot-CoT concatenates the target problem statement with\n"Let\'s think step by step" as an input prompt to LLMs. Despite the success of\nZero-shot-CoT, it still suffers from three pitfalls: calculation errors,\nmissing-step errors, and semantic misunderstanding errors. To address the\nmissing-step errors, we propose Plan-and-Solve (PS) Prompting. It consists of\ntwo components: first, devising a plan to divide the entire task into smaller\nsubtasks, and then carrying out the subtasks according to the plan. To address\nthe calculation errors and improve the quality of generated reasoning ste

In [5]:
link="https://arxiv.org/pdf/2305.04091"
def pdf_scrape(link):
    loader = PyMuPDFLoader(link)
    docs = loader.load()
    pages_content =""
    for doc in docs:
        pages_content += doc.page_content
    return pages_content

pdf_scrape(link)

'Plan-and-Solve Prompting: Improving Zero-Shot Chain-of-Thought\nReasoning by Large Language Models\nLei Wang1 Wanyu Xu2 Yihuai Lan Zhiqiang Hu3 Yunshi Lan4\nRoy Ka-Wei Lee3 Ee-Peng Lim1∗\n1Singapore Management University\n2Southwest Jiaotong University\n3Singapore University of Technology and Design\n4East China Normal University\nAbstract\nLarge language models (LLMs) have recently\nbeen shown to deliver impressive performance\nin various NLP tasks. To tackle multi-step rea-\nsoning tasks, few-shot chain-of-thought (CoT)\nprompting includes a few manually crafted\nstep-by-step reasoning demonstrations which\nenable LLMs to explicitly generate reasoning\nsteps and improve their reasoning task accu-\nracy. To eliminate the manual effort, Zero-\nshot-CoT concatenates the target problem state-\nment with “Let’s think step by step” as an in-\nput prompt to LLMs. Despite the success of\nZero-shot-CoT, it still suffers from three pit-\nfalls: calculation errors, missing-step errors,\nand se

In [6]:
def get_content_from_url(soup):
        """Get the text from the soup

        Args:
            soup (BeautifulSoup): The soup to get the text from

        Returns:
            str: The text from the soup
        """
        text = ""
        tags = ["p", "h1", "h2", "h3", "h4", "h5"]
        for element in soup.find_all(tags):  # Find all the <p> elements
            text += element.text + "\n"
        return text


In [7]:
try:
    link="https://medium.com/@jamesravey/self-hosting-llama-3-on-a-home-server-00feeeba8174"
    session = requests.Session()
    response = session.get(link, timeout=4)
    soup = BeautifulSoup(
        response.content, "lxml", from_encoding=response.encoding
    )

    for script_or_style in soup(["script", "style"]):
        script_or_style.extract()

    raw_content = get_content_from_url(soup)
    lines = (line.strip() for line in raw_content.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    content = "\n".join(chunk for chunk in chunks if chunk)
    content

except Exception as e:
    print("Error! : " + str(e))

content

"Sign up\nSign in\nSign up\nSign in\nSelf-hosting Llama 3 on a home server\nDr James Ravenscroft\nFollow\n--\n2\nListen\nShare\nSelf-hosting Llama 3 as your own ChatGPT replacement service using a 10 year old graphics card and open source components.\nLast week Meta launched Llama 3, the latest in their open source LLM series. Llama 3 is particularly interesting because the 8 billion parameter model, which is small enough to run on a laptop, performs as well as models 10x bigger than it. The responses it provides are as good as GPT-4 for many use cases.\nI finally decided that this was motivation enough to dig out my old Nvidia Titan X card from the loft and slot it into my home server so that I could stand up a ChatGPT clone on my home network. In this post I explain some of the pros and cons of self-hosting llama 3 and provide configuration and resources to help you do it too.\nHow it works\nThe model is served by Ollama which is a GPU-enabled open source service for running LLMs as 

In [8]:

# Download the content
link="https://medium.com/@jamesravey/self-hosting-llama-3-on-a-home-server-00feeeba8174"
def bs_scrape(link):
    response = requests.get(link)
    temp_file = f"temp_{uuid.uuid4()}.html"
    with open(temp_file, "w", encoding="utf-8") as f:
        f.write(response.text)
    # Load it with an HTML parser
    loader = BSHTMLLoader(temp_file)
    document = loader.load()[0]
    if os.path.exists(temp_file):
        os.remove(temp_file)

    # Clean up code
    # Replace consecutive new lines with a single new line
    document.page_content = re.sub("\n\n+", "\n", document.page_content)
    return document.page_content

bs_scrape(link)

'Self-hosting Llama 3 on a home server | by Dr James Ravenscroft | Apr, 2024 | MediumOpen in appSign upSign inWriteSign upSign inSelf-hosting Llama 3 on a home serverDr James Ravenscroft·Follow7 min read·Apr 20, 2024--2ListenShareSelf-hosting Llama 3 as your own ChatGPT replacement service using a 10 year old graphics card and open source components.Last week Meta launched Llama 3, the latest in their open source LLM series. Llama 3 is particularly interesting because the 8 billion parameter model, which is small enough to run on a laptop, performs as well as models 10x bigger than it. The responses it provides are as good as GPT-4 for many use cases.I finally decided that this was motivation enough to dig out my old Nvidia Titan X card from the loft and slot it into my home server so that I could stand up a ChatGPT clone on my home network. In this post I explain some of the pros and cons of self-hosting llama 3 and provide configuration and resources to help you do it too.How it work

In [11]:
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)


link="https://medium.com/@jamesravey/self-hosting-llama-3-on-a-home-server-00feeeba8174"
def web_scrape(link):
    try:
        loader = WebBaseLoader(link)
        loader.requests_kwargs = {"verify": False}
        docs = loader.load()
        content = ""
        for doc in docs:
            content += doc.page_content

        return content

    except Exception as e:
        print("Error! : " + str(e))
        return ""

web_scrape(link)

'Self-hosting Llama 3 on a home server | by Dr James Ravenscroft | Apr, 2024 | MediumOpen in appSign upSign inWriteSign upSign inSelf-hosting Llama 3 on a home serverDr James Ravenscroft·Follow7 min read·Apr 20, 2024--2ListenShareSelf-hosting Llama 3 as your own ChatGPT replacement service using a 10 year old graphics card and open source components.Last week Meta launched Llama 3, the latest in their open source LLM series. Llama 3 is particularly interesting because the 8 billion parameter model, which is small enough to run on a laptop, performs as well as models 10x bigger than it. The responses it provides are as good as GPT-4 for many use cases.I finally decided that this was motivation enough to dig out my old Nvidia Titan X card from the loft and slot it into my home server so that I could stand up a ChatGPT clone on my home network. In this post I explain some of the pros and cons of self-hosting llama 3 and provide configuration and resources to help you do it too.How it work